# Cluster visualisieren und optimieren

Dieses Notebook dient dazu, die entgültigen Verben fürs Spiel zu definieren und feintunen indem wir uns die Cluster in 3D ansehen

In [ ]:
import pandas as pd
import plotly.express as px
from umap import UMAP
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

In [ ]:
synonyms = {
    'go': [
        # Klare Bewegungsverben
        'gehen', 'laufen', 'bewegen', 'kommen',
        'marschieren', 'wandern', 'rennen',
    ],
    'take': [
        # Aufnehmen/Greifen
        'nehmen', 'holen', 'packen', 'greifen', 'aufheben',
        'mitnehmen', 'aufsammeln', 'einsammeln',
    ],
    'drop': [
        # Weglegen/Loslassen
        'ablegen', 'wegwerfen',
        'hinlegen', 'hinstellen',
    ],
    'use': [
        # Benutzen/Anwenden
        'benutzen', 'verwenden', 'anwenden', 'öffnen',
        'einsetzen', 'nutzen', 'gebrauchen', 'bedienen',
    ],
    'examine': [
        # Detailliert untersuchen
        'untersuchen', 'betrachten', 'inspizieren',
        'mustern', 'prüfen', 'begutachten', 'analysieren',
    ],
    'read': [
        # Lesen (Text)
        'lesen', 'durchlesen', 'vorlesen', 'studieren'
    ],
    'talk': [
        # Kommunizieren
        'sprechen', 'reden', 'unterhalten', 
        'quatschen', 'plaudern', 'kommunizieren',
    ],
    'look': [
        # Schnell schauen/umsehen
        'schauen', 'umschauen', 'umsehen', 'gucken', 'ansehen', 'anschauen',
        'blicken', 'spähen',
    ]
}

**Embedding**

In [ ]:
verbs_list = [
    verb 
    for command, verbs in synonyms.items()
    for verb in verbs
]

commands_list = [
    command 
    for command, verbs in synonyms.items()
    for verb in verbs
]

embeddings = model.encode(verbs_list)

**Visualisierung**

UMAP wurde vorgeschlagen und wir probiert weil für globale Ähnlichkeiten UMAP besser funktioniert als t-SNE, was wohl normalisiert und so zwar lokale ähnlichkeiten zeigt aber nicht global, also im gesamten Sprachraum.

In [ ]:
# UMAP konfigurieren
reducer_2d = UMAP(
    n_components=2,
    n_neighbors=15,
    min_dist=0.1,
    random_state=42
)

# Embeddings transformieren
embeddings_2d = reducer_2d.fit_transform(embeddings)

# UMAP konfigurieren
reducer_3d = UMAP(
    n_components=3,
    n_neighbors=15,
    min_dist=0.1,
    random_state=42
)

# Embeddings transformieren
embeddings_3d = reducer_3d.fit_transform(embeddings)

In [ ]:
df_2d = pd.DataFrame({
    'x': embeddings_2d[:, 0],
    'y': embeddings_2d[:, 1],
    'verb': verbs_list,
    'command': commands_list
})

fig_2d = px.scatter(df_2d, x='x', y='y',
                 color='command',
                 # text='verb',
                 hover_data=['verb', 'command'])

df_3d = pd.DataFrame({
    'x': embeddings_3d[:, 0],
    'y': embeddings_3d[:, 2],
    'z': embeddings_3d[:, 1],
    'verb': verbs_list,
    'command': commands_list
})

fig_3d = px.scatter_3d(df_3d, x='x', y='y', z='z',
                 color='command',
                 # text='verb',
                 hover_data=['verb', 'command'])

fig_2d.show()
fig_3d.show()